# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [1]:
import networkx
import urllib2
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [2]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: implement function
    node_list = []
    for line in gfile:
        temp = line.split() 
        if len(temp) != 0:  #check and break when the line is empty
            if temp[0] != '*':  # check to remove first three introduction lines
                node_list.append(temp[0])
        else:
            break
    return node_list
    pass

Next implement a function to read in the edges from the input file.

In [3]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    edge_pair_list = []
    for line in gfile:
        if line[0] == '*': # breaks when it reaches the last line
            break
        temp_list = line[:-1].split(':')[1].split(';') #splits the string into list of connected nodes
        for con_pair in temp_list:
            con_node = con_pair.split(',')
            i = 0
            while i < len(con_node): #pair a node with all other nodes in connected pair array
                j = i +1
                while j < len(con_node):
                    edge_pair_list.append((con_node[i], con_node[j]))
                    j += 1
                i += 1
    return edge_pair_list
    pass

The following code should now correctly create the graph.

In [4]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [5]:
def Search(graph, root):
    """Runs depth-first search through a graph, starting at a given root. Neighboring
    nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    visited = [] # to keep track of all the visited nodes
    
    #gets all descendants of a given node
    def get_descen(node, visited):
        visited.append(node)
        neighbor_list = sorted(list(graph.neighbors(node))) #neighboring nodes in alphabetical order
        if len(neighbor_list) != 0:
            for neighbor in neighbor_list:
                if neighbor not in visited:
                    visited = get_descen(neighbor, visited) #gets descendants of the neighboring element
        return visited
    
    visited = get_descen(root, visited)
    return visited
    pass   

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [6]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [7]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    adjacency_list = list(graph.adjacency()) 
    adjacency_list.sort(key=lambda x: x[0]) # sorting the list in alpabetical order of nodes
    adjacency_list.sort(key = lambda x: len(x[1])) # sorting the list by number of neighboring nodes
    connected_components_list = []
    visited = [] # to keep track of the visited nodes
    
    for node in adjacency_list:
        if node[0] not in visited:
            connected_tree = Search(graph,node[0]) # get the connected component of a given node
            connected_components_list.append(connected_tree) 
            visited += connected_tree
    
    return connected_components_list
    pass

We will check correctness of your code by verifying that your output list is identical to our solution.

In [8]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [9]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True


In [10]:
character_interactions

[['7Q'],
 ['91'],
 ['99'],
 ['9H'],
 ['9P'],
 ['08',
  '07',
  '0F',
  '0G',
  '0H',
  '0I',
  '0J',
  '0K',
  '3V',
  'AC',
  '0M',
  '0L',
  '2I',
  'AH',
  '2J',
  'HT',
  '02',
  '01',
  'AE',
  '31',
  'AO',
  '3T',
  '4C',
  'AI',
  'AF',
  'AA',
  '0N',
  '0O',
  '0P',
  '0Q',
  '0R',
  '0S',
  'AN',
  '14',
  '15',
  'MG',
  '4Z',
  '6G',
  '6H',
  '7R',
  '7S',
  '7T',
  '9W',
  '9V',
  '9X',
  'AL',
  '06',
  '3W',
  '3X',
  '3Y',
  '3Z',
  '40',
  '93',
  '44',
  '45',
  '46',
  '92',
  '94',
  '22',
  '41',
  '42',
  '43',
  '95',
  '6Y',
  'NE',
  '5A',
  '57',
  'PO',
  '1K',
  '79',
  'EO',
  '2A',
  '2B',
  '2C',
  '2D',
  '2E',
  'AM',
  '12',
  'ME',
  '03',
  '04',
  '05',
  'DI',
  '0Y',
  '0Z',
  '10',
  'HP',
  '90',
  'TH',
  '5D',
  'PE',
  '5L',
  '5N',
  '5O',
  '5E',
  '5F',
  '5K',
  '5G',
  '5H',
  '5I',
  '5J',
  'HM',
  'AR',
  '2F',
  'BL',
  '29',
  '83',
  'ZE',
  '1F',
  '1G',
  'OG',
  '87',
  'DT',
  'SE',
  'AP',
  '2Z',
  '53',
  '7F',
  '7G',
  '